## Coding Attention Mechanism ##

BAB 3  
1. Alasan mengapa menggunakan attention mechanism di neural networks
2. Dasar self-attention, hingga self-attention lanjut
3. Causal attention module yang menjadikan LLM generate satu token pada satu waktu
4. Menghindari overfitting dengan cara masking random
5. Menumpuk beberapa attention hingga menjadi multi-head attention

Beberapa metode Attention :  
1. Simplified self-attention : introduce the border ide
2. Self-attention : self-attention dengan bobot yang trainable dan merupakan basis yang digunakan dalam LLMs
3. Causal attention : menjadikan model untuk hanya mempertimbangkan input sebelumnya dan input saat ini pada urutan
4. Multi-head attention : secara simultan menghitung informasi dari representasi yang berbeda (multi self attention)

### 3.1 The problem with modeling long sequences

Ambil contoh kita ingin membuat sebuah model penerjemah dari dua bahasa yang berbeda. Tidak mungkin kita menerjemahkan bahasa secara kata demi kata, karena menerjemahkan bahasa memerlukan pemahaman konteks dan grammar. 

Untuk mengatasi masalah ini, sangat umum menggunakan deep neural networks dengan dua submodules yaitu *encoder* dan *decoder*.

Sebelum ditemukannya Tranformers, Recurrent Neural Networks (RNN) adalah algoritma yang paling populer untuk arsitektur *encoder-decoder* dalam kasus terjemahan. 

Dalam sebuah RNN encoder-decoder, teks input dimasukkan ke dalam encoder, yang memprosesnya secara berurutan. Encoder memperbarui keadaan tersembunyinya (nilai internal pada lapisan tersembunyi) di setiap langkah, berusaha menangkap seluruh makna kalimat input dalam keadaan tersembunyi terakhir, seperti yang diilustrasikan pada gambar 3.4. Decoder kemudian mengambil keadaan tersembunyi terakhir ini untuk mulai menghasilkan kalimat terjemahan, satu kata pada satu waktu. Decoder juga memperbarui keadaan tersembunyinya di setiap langkah, yang seharusnya membawa konteks yang diperlukan untuk prediksi kata berikutnya.

Tidak dapat mengakses secara langsung keadaan tersembunyi sebelumnya dari encoder selama fase decoding. Akibatnya, ia hanya bergantung pada keadaan tersembunyi saat ini, yang merangkum semua informasi yang relevan. Hal ini dapat menyebabkan hilangnya konteks, terutama dalam kalimat kompleks di mana ketergantungan mungkin mencakup jarak yang jauh.

### 3.2 Capturing data dependencies with attention mechanisms

Bahdanau Attention Mechanism (2014) memodifikasi *encoder-decoder* RNN, dimana *decoder* dapat dengan selektif mengakses bagian yang berbeda dari urutan input pada setiap proses decoding. Pada teks generator, *decoder* dapat dengan selektif mengakses input token. Ini berarti beberapa token lebih penting daripada yang lain untuk membuat sebuah output token. Seberapa penting ini ditentukan oleh **bobot perhatian/attention weights**

### 3.3 Attending to different parts of the input with self-attention

#### 3.3.1 A simple self-attention mehcanism without trainable weights

Membuat sebuah mekanisme self-attention sederhana tanpa weights. Nah, goals dari self-attention ini adalah untuk menghitung konteks vektor untuk setiap elemen input yang mengkombinasi informasi dari semua input yang lain

Self-attention akan mencari nilai attention-scores antar token input. Sebagai contoh AS12 - attention score antara X1 dan X2. X1, X2,...XN adalah sebuah token embedding dengan n_dimensional vector. Attention score akan menghasilkan skalar (satu nilai) 

In [1]:
import torch

In [3]:
# token embeddings
# with dimensions 6x3 (6 tokens, 3 dimensions)
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x1)
     [0.55, 0.87, 0.66], # journey (x2)
     [0.57, 0.85, 0.64], # starts (x3)
     [0.22, 0.58, 0.33], # with (x4)
     [0.77, 0.25, 0.10], # one (x5)
     [0.05, 0.80, 0.55] # step (x6)
    ]
)

- query token : token yang ingin dicari nilai attention scorenya terhadap token-token lain

In [4]:
query = inputs[1] # second inputs (x2/journey) as the query

attn_scores_2 = torch.empty(inputs.shape[0]) # shape[0] : 6 (number of tokens)

# loop ini akan menghitung dot antara query dan setiap token
# kemudian dimasukkan ke dalam attn_scores_2
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i,query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


**NB** : Dot product akan menghasilkan scalar. Nilai dari dot product ini akan memberikan nilai/arti kesamaan (similiarty), karena menjelaskan bagaimana dekatnya dua vektor yang sejajar. Dalam konteks self-attention nilai dot product menentukan sejauh mana setiap elemen dalam suatu urutan berfokus pada, atau “memperhatikan,” elemen lainnya: semakin tinggi produk titik, semakin tinggi pula skor kesamaan dan perhatian antara dua elemen.

Next step adalah menormalisasi setiap attention-score sehingga ketika dijumlahkan hasilnya 1. Ini berguna untuk kestabilan dalam pelatihan LLM.

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print('Attention weights : ', attn_weights_2_tmp)
print('Sum : ', attn_weights_2_tmp.sum())

Attention weights :  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum :  tensor(1.0000)


In [6]:
# softmax
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print('Attention weights (softmax naive) : ', attn_weights_2_naive)
print('Sum : ', attn_weights_2_naive.sum())

Attention weights (softmax naive) :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print('Attention weights (softmax) : ', attn_weights_2)
print('Sum : ', attn_weights_2.sum())

Attention weights (softmax) :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


Last step menghitung context vector (z) dengn cara mengkalilkan input token embedding (x) dengan hasil dari attention-weights dan kemudian menjumlahkan vectornya.

In [8]:
query = inputs[1] # second inputs (x2/journey) as the query
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


Hasil diatas adalah hasil context vector kedua karena attention-weights computed with respect to the secod input vector in the previous step

#### 3.3.2 Computing attention weights for all input tokens

1) Compute attention scores
2) Compute attention weights
3) Compute context vector

In [9]:
# 1
attn_scores = torch.empty(6, 6) # same as attn_scores_2 but for all tokens
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i][j] = torch.dot(x_i,x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


Hasil diatas merupakan representasi dari attention-score antara setiap pasangan input

In [10]:
# 2
# normalize
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [11]:
# 3
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


### 3.4 Implementing self-attention with trainable weights

Mekanisme self-attention yang paling popouler digunakan adalah **scaled dot-product attention**

#### 3.4.1 Computing the attention weights step by step

In [12]:
x_2 = inputs[1] # second inputs (x2/journey) as the query
d_in = inputs.shape[1] # 3
d_out = 2 # the output embedding size

In [13]:
# initialize three weight matrices (Wq, Wk, Wv)
torch.manual_seed(123) # for reproducibility
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [14]:
# compute query, key, value vectors
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print('Query : ', query_2)
print('Key : ', key_2)  
print('Value : ', value_2)

Query :  tensor([0.4306, 1.4551])
Key :  tensor([0.4433, 1.1419])
Value :  tensor([0.3951, 1.0037])


In [15]:
# obtain all keys and values
keys = inputs @ W_key
values = inputs @ W_value
print('Keys shape : ', keys.shape)
print('Values shape : ', values.shape)

Keys shape :  torch.Size([6, 2])
Values shape :  torch.Size([6, 2])


Compute attention-score

In [16]:
# first, compute attention-scores 22
keys_2 = keys[1] # second input 
attn_score_22 = query_2.dot(keys_2) # dot product between query and key
print(attn_score_22)

tensor(1.8524)


In [17]:
# computation all attention score via matrix multiplication

attn_scores_2 = query_2 @ keys.T # all attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


Sekarang kita berlanjut dari attention-scores menjadi attention-weights. Kita menghitung attentionn-weights dengan menggunakan softmax function. Untuk saat ini kita membagi attention-scores dengan akar kuadrat dari dimensi key

In [18]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


NB : Alasan mengapa dilakukan normalisasi dengan dimensi embedding adalah untuk meningkatkan performa pelatihan dengan menghindari gradient yang sangat kecil. 

Last step adalah menghitung context-vector

In [19]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


In [20]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vect = attn_weights @ values
        return context_vect

In [21]:
torch.manual_seed(123)

sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


Alih-alih memakai nn.Parameter(torch.rand()) lebih baik menggunakan nn.Linear, karena memiliki optimasi yang baik dalam inisialisasi bobot yang dapat berkontribusi pada pelatihan yang lebih efektif dan stabil

In [22]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vect = attn_weights @ values
        return context_vect

In [23]:
torch.manual_seed(789)

sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


### 3.5 Hiding future words with causal attention

Kebanyakan tugas dari LLM, meminta self-attention mechanism untuk hanya mempertimbangkan token yang posisinya muncul sebelum posisi saat ini ketika memprediksi token selanjutnya pada sebuah urutan.

Masked-attention adalah spesialisasi dari self attention, dimana model hanya akan mempertimbangkan input sebelumnya dan input saat ini pada sebuah urutan untuk mempertimbangkan attention score.

#### 3.5.1 Applying a causal attention mask

Key idea untuk memperoleh masked-attention weight pada causal-attention adalah melakukan normalisasi pada attention scores, dan memberikan nilai 0 pada elemenent-element diatas diagonal utama dan melakukan normalisasi

In [24]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [25]:
context_length = attn_scores.shape[0]

mask_simple = torch.tril(torch.ones(context_length, context_length), diagonal=0)
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [26]:
# multiply attn_weights with mask_simple
# to zero-out the value
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [27]:
# renormalize
# sum up to 1 in each row
row_sums  = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


Cara yang lebih efisien untuk mendapatkan masked attention weight pada causal attention adalah melakukan masking pada attention-scores dengan nilai negatif infinity sebelum menggunakan fungsi softmax.

In [28]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [29]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


Setelah ini bisa menggunakan attention weights yang telah dimodifikasi untuk menghitung context-vector via `context_vec = attn_weights @ values`

#### 3.5.2 Masking additional attention weights with dropout

Dropout pada deep learning adalah sebuah teknik dimana secara random memilih unit hidden layer untuk diabaikan selama pelatihan, secara efektif "dropping out". Metode ini membantu untuk mencegah overfitting, sehingga model tidak terlalu bergantung pada semua hidden layer secara spesifik. 

In [30]:
torch.manual_seed(123)

dropout = torch.nn.Dropout(0.5) # 50% dropout rate - a half of matrix randomly set to zero
example = torch.ones(6, 6)
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [31]:
torch.manual_seed(123)

print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


#### 3.5.3 Implementing a compact causal attention class

In [32]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


Hasilnya adalah sebuah tensor 3 dimensi dengan komposisi 2 input teks, 6 token setiap teks, dan setiap token adalah vektor embedding 3 dimensi

In [33]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias = False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vector = attn_weights @ values

        return context_vector

`register_buffer` berarti secara otomatis buffer dipindahlan ke perangkat yang sesuai (CPU atau GPU) bersamaan dengan model kita. Ini berarti kita tidak perlu memastikan tensor ini secara manual berada di perangkat yang sama dengan parameter model.

In [34]:
torch.manual_seed(123)

context_length = batch.shape[1]

ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print("Context_vecs.shape : ", context_vecs.shape)

Context_vecs.shape :  torch.Size([2, 6, 2])


Hasilnya adalah context vector 3 dimensi dimana setiap token direpresentasikan dengan vector 2 dimensi

### 3.6 Extending single-head attention to multi_head attention

Multi-head attention berarti meng-expand Causal-attention kita menjadi beberapa 'head' dan beroperasi secara individu.

#### 3.6.1 Stacking multiple single-head attention layers

In [35]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        self.heads = nn.ModuleList(CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads))

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [36]:
torch.manual_seed(123)

context_length = batch.shape[1] # number of tokens
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print("Context_vecs.shape : ", context_vecs.shape)
print("Context_vecs : ", context_vecs)

Context_vecs.shape :  torch.Size([2, 6, 4])
Context_vecs :  tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


Dari hasil, tensor ini berdimensi 3 dengan komponen 2 teks, 6 tokens setiap teks, dan 4 dimensional vektor tiap token

#### 3.6.2 Implementing multi-head attention with weight splits

Daripada kita me-maintanance dua kelas yang berbeda, kita akan menggabungkan dua konsep menjadi single MultiHeadAttention. CausakAttention memproses secara independen dan menggabungkan semuanya menjadi satu. Pada proses ini kita akan membagi menjadi beberapa kepala dengan mengubah bentuk  tensor **projected query, key, dan value** dan menggabungkannya setelah menghitung attention.

In [43]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # size of each attention-head
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) # combine head output 
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        # tensor shape : (b, num_tokens, d_out)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        # secara implisit kita split matriks dengan menambah num_heads 
        # secara konseptual kita sedang memecah representasi embedding pada setiap token menjadi num_heads bagian

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        # kita menukar dimensi ke-1 dan ke-2 dari tensor keys, queries, dan values
        # menjadi :  (b, num_heads, num_tokens, head_dim)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        context_vec = self.out_proj(context_vec)

        return context_vec

In [38]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],   
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])
# tensor with shape (b, num_heads, num_tokens, head_dim)

In [ ]:
# perform a batched matrix multiplication
# transpose the last two dimensions of a
print(a @ a.transpose(2, 3))

# the output have first head and second head in the same batch

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


Semua context-vector dari semua heads ditranspose kembali ke dalam bentuk (b, num_tokens, num_heads, head_dim). Tensor ini kemudian di-reshape (flattened) menjadi (b, num_tokens, d_out), secara efektif menggabungkan semua output dari setiap heads.  
Selain itu, juga ditambahkan output projection layer (self.out_proj) setelah menggabungkan heads. 

In [44]:
# the MultiHeadAttention class can be used similiar to the SelfAttention and CausalAttention class
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print('Context vecs shape : ', context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
Context vecs shape :  torch.Size([2, 6, 2])


Dari output dapat dilihat bahwa dimensi dari output itu sendiri secara langsung dikontrol oleh `d_out`